<a href="https://colab.research.google.com/github/KML-Fig09/ComputerPeripherals/blob/FirstBranch/FINAL_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1 - Set up environment
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Install dependencies
!pip install -r requirements.txt


fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [2]:
#Step 2 - Install Dataset!

# Install the roboflow library
!pip install roboflow

# Import roboflow and configure the dataset download
from roboflow import Roboflow

rf = Roboflow(api_key="vrgniXrCoM7OOUUcvkeu")
project = rf.workspace("mouse-2lugf").project("raava")
dataset = project.version(1).download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Raava-1 in yolov5pytorch:: 100%|██████████| 3156/3156 [00:00<00:00, 8225.40it/s]


Note: after running the above steps, you will need to restart the runtime and rerun them to continue with the most up-to-date libraries,

In [3]:
# Step 3 - Write CBAM file
%%writefile /content/yolov5/models/cbam.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, relu=True):
        super(BasicConv, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False)
        self.bn = nn.BatchNorm2d(out_planes, eps=1e-5, momentum=0.01, affine=True)
        self.relu = nn.ReLU(inplace=True) if relu else None

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.relu:
            x = self.relu(x)
        return x

class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
        )
        self.pool_types = pool_types

    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type == 'avg':
                avg_pool = F.avg_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp(avg_pool.view(avg_pool.size(0), -1))
            elif pool_type == 'max':
                max_pool = F.max_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp(max_pool.view(max_pool.size(0), -1))
            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = torch.sigmoid(channel_att_sum).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1), torch.mean(x, 1).unsqueeze(1)), dim=1)

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1)//2, relu=False)

    def forward(self, x):
        x_compress = self.compress(x)
        scale = torch.sigmoid(self.spatial(x_compress))
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial = no_spatial
        if not self.no_spatial:
            self.SpatialGate = SpatialGate()

    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out


Writing /content/yolov5/models/cbam.py


In [4]:
#Step 4 - Backup and read content of common.py file just in case

# Read the content of common.py
common_py_path = '/content/yolov5/models/common.py'
with open(common_py_path, 'r') as file:
    common_py_content = file.read()

# Create a backup of common.py
backup_common_py_path = '/content/yolov5/models/common.py.bak'
with open(backup_common_py_path, 'w') as file:
    file.write(common_py_content)

print("Backup of common.py created at /content/yolov5/models/common.py.bak")


Backup of common.py created at /content/yolov5/models/common.py.bak


In [5]:
# Step 5 - add CBAM module directly to the common.py file
common_py_path = '/content/yolov5/models/common.py'

with open(common_py_path, 'r') as file:
    common_py_content = file.read()

# Modify common.py to include the CBAM module
common_py_content = common_py_content.replace(
    "class C3(nn.Module):",
    "from models.cbam import CBAM\n\nclass C3(nn.Module):"
)

c3_class_start = common_py_content.find("class C3(nn.Module):")
c3_class_end = common_py_content.find("def forward(self, x):", c3_class_start) + len("def forward(self, x):")
c3_class_body = common_py_content[c3_class_start:c3_class_end]

c3_class_body = c3_class_body.replace(
    "self.cv3 = Conv(2 * c_, c2, 1)",  # Optional shortcut
    "self.cv3 = Conv(2 * c_, c2, 1)\n        self.cbam = CBAM(c2)"
)
c3_class_body = c3_class_body.replace(
    "return self.cv3(torch.cat((self.m(self.cv1(x)), self.cv2(x)), 1))",
    "y1 = self.cv1(x)\n        y2 = self.cv2(x)\n        y = torch.cat((self.m(y1), y2), 1)\n        y = self.cv3(y)\n        return self.cbam(y)"
)

common_py_content = common_py_content[:c3_class_start] + c3_class_body + common_py_content[c3_class_end:]

with open(common_py_path, 'w') as file:
    file.write(common_py_content)

print("common.py has been modified to include the CBAM module.")


common.py has been modified to include the CBAM module.


Note: the above step makes these modifications:

  Add the following import at the top:

`from models.cbam import CBAM`

  Plus, add the CBAM module to the C3 class, will look like:

```
class C3(nn.Module):
    # CSP Bottleneck with 3 convolutions
    def __init__(self, c1, c2, n=1, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super(C3, self).__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c1, c_, 1, 1)
        self.cv3 = Conv(2 * c_, c2, 1)  # optional shortcut
        self.m = nn.Sequential(*(Bottleneck(c_, c_, shortcut, g, e=1.0) for _ in range(n)))
        self.cbam = CBAM(c2)  # Add CBAM here

    def forward(self, x):
        return self.cv3(torch.cat((self.m(self.cv1(x)), self.cv2(x)), 1)) * self.cbam(x)  # Apply CBAM here
```


Before the next step, open the data.yaml file in the Raava-1 dataset and ensure that the test/train/validation file paths are all correct. They should read like this:

```
test: /content/yolov5/Raava-1/test/images
train: /content/yolov5/Raava-1/train/images
val: /content/yolov5/Raava-1/valid/images

# class weights (optional)
weights: [2.5, 2.5, 1, 1, 0.9, 1.0, 3.5, 1.0, 1.0]  # Higher weights for "Keyboard" and "Mouse", lower for "headset"

```



In [7]:
# Step 6 - Add necessary imports and modifications to train.py
train_py_path = '/content/yolov5/train.py'

with open(train_py_path, 'r') as file:
    train_py_content = file.read()

train_py_content = train_py_content.replace(
    "import torch.optim as optim",
    """import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.nn as nn"""
)

optimizer_code = """
# Define the optimizer directly as Adam
optimizer = optim.Adam(model.parameters(), lr=hyp["lr0"], weight_decay=hyp["weight_decay"])
"""

train_py_content = train_py_content.replace(
    "optimizer = smart_optimizer(model, opt.optimizer, hyp['lr0'], hyp['momentum'], hyp['weight_decay'])",
    optimizer_code
)

class_weights_code = """
 Calculate class weights
class_labels = [label for _, label in train_dataset]
class_weights = compute_class_weight('balanced', classes=np.unique(class_labels), y=class_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define a custom loss function to use class weights
class WeightedLoss(nn.Module):
    def __init__(self, model, class_weights):
        super(WeightedLoss, self).__init__()
        self.model = model
        self.class_weights = class_weights

    def forward(self, imgs, targets):
        loss, _ = self.model(imgs, targets)
        class_loss = nn.CrossEntropyLoss(weight=self.class_weights)
        weighted_loss = loss + class_loss(targets)
        return weighted_loss

# Use the custom weighted loss function during training
criterion = WeightedLoss(model, class_weights)
#"""

train_py_content = train_py_content.replace(
    "criterion = ComputeLoss(model)",
    class_weights_code
)

with open(train_py_path, 'w') as file:
    file.write(train_py_content)

print("train.py has been modified to use the Adam optimizer and handle class imbalance using class weights.")


train.py has been modified to use the Adam optimizer and handle class imbalance using class weights.


In [ ]:
# Step 7 - Create change dominant colours functions
# Please note, this step was originally used as a set up for a colour change function written inside the detect.py file - though I'm not using this method anymore, I wanted to include a snippet of what it looked like originally.
import cv2
import numpy as np
from sklearn.cluster import KMeans

def get_dominant_colors(image, k=4):
    # Resize image to speed up processing
    image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
    data = np.reshape(image, (-1, 3))
    kmeans = KMeans(n_clusters=k, random_state=0).fit(data)
    colors = kmeans.cluster_centers_.astype(int)
    return colors


In [16]:
# Step 8 - Train the model!

# Train the model (make sure to specify your dataset path correctly)
!python train.py --img 640 --batch 16 --epochs 40 --data {dataset.location}/data.yaml --weights /content/yolov5/runs/train/exp/weights/best.pt --cache --optimizer Adam --cos-lr


2024-06-11 23:45:42.881861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 23:45:42.881916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 23:45:42.883522: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/yolov5/runs/train/exp/weights/best.pt, cfg=, data=/content/yolov5/Raava-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights

In [17]:
#Step 9 - Inference (testing)

# Run inference with the trained model
!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 640 --conf 0.20 --source /content/ICY_SETUP_2.mp4



detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/ICY_SETUP_2.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 245 layers, 7132162 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/2832) /content/ICY_SETUP_2.mp4: 384x640 (no detections), 105.5ms
video 1/1 (2/2832) /content/ICY_SETUP_2.mp4: 384x640 (no detections), 7.0ms
video 1/1 (3/2832) /content/ICY_SETUP_2.mp4: 384x640 (no detections), 6.5ms
video 1/1 (4/2832) /content/ICY_SETUP_2.mp4: 384x640 (no detections), 6.2ms
v

In [15]:
#Step 10 - Further installations before 10.5
!pip install opencv-python-headless
!pip install torch torchvision torchaudio
!pip install pandas
!pip install seaborn

In [ ]:
#Step 10.5 - alter bounding boxes

import os
import cv2
import torch
import numpy as np
from collections import Counter
from pathlib import Path

# Set paths
YOLOV5_PATH = "/content/yolov5/"
MODEL_PATH = "/content/yolov5/runs/train/exp2/weights/best.pt"
VIDEO_PATH = "/content/ICY_SETUP_2.mp4"

# Load YOLOv5 model
model = torch.hub.load(YOLOV5_PATH, 'custom', path=MODEL_PATH, source='local')

# Function to get the most common color in the image
def most_common_color(image):
    pixels = np.float32(image.reshape(-1, 3))
    n_colors = 5
    _, counts = np.unique(pixels, return_counts=True, axis=0)
    dominant = pixels[counts.argmax()]
    return dominant.astype(int)

# Process video
cap = cv2.VideoCapture(VIDEO_PATH)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = VIDEO_PATH.replace('.mp4', '_output.mp4')

# Ensure the video size is 852x480
out = cv2.VideoWriter(output_path, fourcc, 20.0, (852, 480))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame if necessary
    if frame.shape[1] != 852 or frame.shape[0] != 480:
        frame = cv2.resize(frame, (852, 480))

    # Perform inference
    results = model(frame, size=640)
    results = results.pandas().xyxy[0]  # Pandas DataFrame

    # Get most common color in the frame
    color = most_common_color(frame)
    color = (int(color[0]), int(color[1]), int(color[2]))  # Convert to BGR format

    # Draw bounding boxes
    for _, row in results.iterrows():
        cv2.rectangle(frame, (int(row['xmin']), int(row['ymin'])), (int(row['xmax']), int(row['ymax'])), color, 2)
        cv2.putText(frame, row['name'], (int(row['xmin']), int(row['ymin']) - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, color, 3)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()
print(f"Video processed and saved as {output_path}")


YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 245 layers, 7132162 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [23]:
#This code is for zipping and downloading files as and when needed, not necessary for the model but useful to have.

from google.colab import files
import zipfile
import os

# Function to zip a directory
def zip_directory(folder_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(folder_path, '..')))
    print(f"Directory {folder_path} zipped successfully into {zip_name}.")

# Function to download the zipped file
def download_file(file_path):
    files.download(file_path)
    print(f"File {file_path} is ready for download.")

# Example usage
# Specify the directory path and the name of the zip file
folder_path = '/content/yolov5/runs/detect'  # Change this to the desired directory path
zip_name = 'detect_exp3.zip'

# Zip the directory
zip_directory(folder_path, zip_name)

# Download the zipped file
download_file(zip_name)


Directory /content/yolov5/runs/detect zipped successfully into detect_exp3.zip.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File detect_exp3.zip is ready for download.


In [19]:
#This step is for downloading single files
from google.colab import files
import zipfile
import os

# Function to download the file
def download_file(file_path):
    files.download(file_path)
    print(f"File {file_path} is ready for download.")

download_file('/content/variables.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File /content/variables.pkl is ready for download.


In [18]:
#If you wish to SAVE a pickle file locally.
import pickle

# Save variables to a pickle file
variables = {
    'rf': rf,
    'project': project,
    'dataset': dataset,
    'common_py_content': common_py_content,
    'train_py_content': train_py_content,
}

with open('/content/variables.pkl', 'wb') as f:
    pickle.dump(variables, f)


In [20]:
#If you wish to SAVE a pickle file in Drive.
from google.colab import drive
drive.mount('/content/drive')

# Copy the pickle file to Google Drive
import shutil
shutil.copy('/content/variables.pkl', '/content/drive/My Drive/variables.pkl')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/variables.pkl'

In [21]:
#If you wish to LOAD a pickle file
# Load variables from the pickle file
import pickle

with open('/content/variables.pkl', 'rb') as f:
    variables = pickle.load(f)

rf = variables['rf']
project = variables['project']
dataset = variables['dataset']
common_py_content = variables['common_py_content']
train_py_content = variables['train_py_content']

print("Variables loaded successfully.")


Variables loaded successfully.


In [22]:
#If you wish to LOAD a pickle file from drive.
from google.colab import drive
drive.mount('/content/drive')

# Copy the pickle file from Google Drive
import shutil
shutil.copy('/content/drive/My Drive/variables.pkl', '/content/variables.pkl')

# Load variables from the pickle file
import pickle

with open('/content/variables.pkl', 'rb') as f:
    variables = pickle.load(f)

rf = variables['rf']
project = variables['project']
dataset = variables['dataset']
common_py_content = variables['common_py_content']
train_py_content = variables['train_py_content']

print("Variables loaded successfully.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Variables loaded successfully.
